In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import seaborn as sns
sns.set_theme(style="whitegrid")
import statistics
import math
from scipy import stats
import json
from pandas import json_normalize
import xmltojson
import json
import requests

In [19]:
base_url_pov = 'https://api.census.gov/data/timeseries/poverty/histpov2/geography.html'

html_response = requests.get(base_url_pov)
  
# Save the page content as sample.html
with open("poverty.html", "w") as html_file:
    html_file.write(html_response.text)

In [20]:
with open("poverty.html", "r") as html_file:
    html = html_file.read()
    pov_json = xmltojson.parse(html)

ExpatError: syntax error: line 1, column 56

In [ ]:
with open("data.json", "w") as file:
    json.dump(pov_json, file)

In [ ]:
req1 = requests.get(base_url_pov)
print(req1)  # response 200!

pov_data = req1.json()

In [4]:
base_url_wat = 'https://enviro.epa.gov/enviro/efservice/SDW_CONTAM_VIOL_CITY/PWSID/IL3141937/JSON'
req2 = requests.get(base_url_wat)
print(req2)  # response 200!

json_data_wat = req2.json()
print(json_data_wat)

<Response [200]>
[{'PWSID': 'IL3141937', 'PWSNAME': 'TIMBER PNTE OTDR CNTR-POOL BHS', 'STATE': 'IL', 'COUNTYSERVED': 'MCLEAN', 'GEOLOCATION_CITY': 'MERNA', 'VIOID': 1, 'CCODE': 3100, 'CNAME': 'Coliform (TCR)', 'SOURCES': 'Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and animal fecal waste.', 'DEFINITION': 'Coliforms are bacteria that are naturally present in the environment and used as an indicator that other, potentially harmful, bacteria may be present. Coliforms found in more samples than allowed is a warning of potential problems.', 'HEALTH_EFFECTS': 'Not a health threat in itself; it is used to indicate whether other potentially harmful bacteria may be present5', 'CTYPE': 'TCR', 'VCODE': 22, 'VNAME': 'MCL, Monthly (TCR)', 'VTYPE': 'MCL', 'VIOLMEASURE': 'Maximum Contaminant Level Violation', 'ENFACTIONTYPE': 'SFJ', 'ENFACTIONNAME': 'St Formal NOV issued', 'ENFDATE': '19-JUL-10', 'COMPPERBEGINDATE': '01-APR-10

In [5]:
# Serializing json 
json_object_wat = json.dumps(json_data_wat)
  
# Writing to .json
with open("water.json", "w") as outfile:
    outfile.write(json_object_wat)

In [10]:
# load data using Python JSON module
with open("water.json","r") as f:
    water_data = json.loads(f.read())

# Reading data
water_df = pd.DataFrame(water_data)
water_df.head()

,PWSID,PWSNAME,STATE,COUNTYSERVED,GEOLOCATION_CITY,VIOID,CCODE,CNAME,SOURCES,DEFINITION,...,CTYPE,VCODE,VNAME,VTYPE,VIOLMEASURE,ENFACTIONTYPE,ENFACTIONNAME,ENFDATE,COMPPERBEGINDATE,COMPPERENDDATE
0,IL3141937,TIMBER PNTE OTDR CNTR-POOL BHS,IL,MCLEAN,MERNA,1,3100,Coliform (TCR),Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,...,TCR,22,"MCL, Monthly (TCR)",MCL,Maximum Contaminant Level Violation,SFJ,St Formal NOV issued,19-JUL-10,01-APR-10,30-JUN-10
1,IL3141937,TIMBER PNTE OTDR CNTR-POOL BHS,IL,MCLEAN,MERNA,1,3100,Coliform (TCR),Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,...,TCR,22,"MCL, Monthly (TCR)",MCL,Maximum Contaminant Level Violation,SIF,St Public Notif received,19-JUL-10,01-APR-10,30-JUN-10
2,IL3141937,TIMBER PNTE OTDR CNTR-POOL BHS,IL,MCLEAN,GRIDLEY,1,3100,Coliform (TCR),Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,...,TCR,22,"MCL, Monthly (TCR)",MCL,Maximum Contaminant Level Violation,SFJ,St Formal NOV issued,19-JUL-10,01-APR-10,30-JUN-10
3,IL3141937,TIMBER PNTE OTDR CNTR-POOL BHS,IL,MCLEAN,GRIDLEY,1,3100,Coliform (TCR),Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,...,TCR,22,"MCL, Monthly (TCR)",MCL,Maximum Contaminant Level Violation,SIF,St Public Notif received,19-JUL-10,01-APR-10,30-JUN-10
4,IL3141937,TIMBER PNTE OTDR CNTR-POOL BHS,IL,MCLEAN,BELLFLOWER,1,3100,Coliform (TCR),Coliforms are naturally present in the environ...,Coliforms are bacteria that are naturally pres...,...,TCR,22,"MCL, Monthly (TCR)",MCL,Maximum Contaminant Level Violation,SFJ,St Formal NOV issued,19-JUL-10,01-APR-10,30-JUN-10
